In [1]:
#!pip install recommenders[examples] --user

In [1]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k, rmse
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.
System version: 3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]
PyTorch version: 1.8.1+cpu
Cornac version: 1.14.2


In [26]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [14]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    header=["userID", "itemID", "rating"]
)

data.head()

100%|██████████| 4.81k/4.81k [00:02<00:00, 1.63kKB/s]


,userID,itemID,rating
0,196,242,3.0
1,186,302,3.0
2,22,377,1.0
3,244,51,2.0
4,166,346,1.0


---

In [8]:
import pandas as pd
df = pd.read_csv("songsDataset.csv")
#df = pd.read_csv("dew_df.csv")

In [36]:
#df.drop(["Unnamed: 0"], inplace=True, axis=1)

In [9]:
df

,'userID','songID','rating'
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
...,...,...,...
1999995,199999,75840,2
1999996,199999,85921,1
1999997,199999,116763,1
1999998,199999,128758,1


In [38]:
#df.rename(columns={"userId": "userID","productId": "itemID","Rating": "rating"},inplace=True)

In [39]:
#new_df=df.groupby("userID").filter(lambda x:x['rating'].count() >=2000)

In [40]:
#new_df.shape

(0, 3)

In [10]:
df.columns = df.columns.str.replace('\'','')

In [11]:
df.head()

,userID,songID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5


In [12]:
df.rename(columns={"songID": "itemID"},inplace=True)

In [13]:
df

,userID,itemID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
...,...,...,...
1999995,199999,75840,2
1999996,199999,85921,1
1999997,199999,116763,1
1999998,199999,128758,1


In [16]:
data = df

In [37]:
#data = data[data["userID"]<2000]

In [17]:
data

,userID,itemID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
...,...,...,...
1999995,199999,75840,2
1999996,199999,85921,1
1999997,199999,116763,1
1999998,199999,128758,1


---

In [18]:
data

,userID,itemID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
...,...,...,...
1999995,199999,75840,2
1999996,199999,85921,1
1999997,199999,116763,1
1999998,199999,128758,1


In [19]:
data.groupby(['userID'])['itemID'].count().max()

10

In [20]:
data.groupby(['userID'])['itemID'].count().min()

10

In [21]:
data

,userID,itemID,rating
0,0,7171,5
1,0,8637,4
2,0,21966,4
3,0,35821,5
4,0,82446,5
...,...,...,...
1999995,199999,75840,2
1999996,199999,85921,1
1999997,199999,116763,1
1999998,199999,128758,1


---

In [83]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
data_s = Dataset.load_from_df(data, reader)
svd = SVD()
result = cross_validate(svd, data_s, measures=['RMSE', 'MAE'],cv=5)
print(result['test_rmse'].mean())

1.2709758762574144


In [84]:
from surprise.model_selection import KFold
from surprise import Dataset, SVD

In [85]:
from collections import defaultdict
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [86]:
kf = KFold(n_splits=5)
algo = SVD()

for trainset, testset in kf.split(data_s):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4.9)

    # Precision and recall can then be averaged over all users
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print(sum(rec for rec in recalls.values()) / len(recalls))

0.005381043466445075
0.004430563182777791
0.00539754262042178
0.004505479229262413
0.005190108264249313
0.004326233953297666
0.005381913109895657
0.004448317939623174
0.005511741064195635
0.004643571923946342


In [22]:
import gc
gc.collect()

538

---

In [23]:
train, test = python_random_split(data, 0.75)

In [27]:
train, test = python_random_split(data, 0.999)

In [28]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 200000
Number of items: 127753


In [29]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 13.1468 seconds for prediction.


In [ ]:
all_predictions.groupby(['userID'])['itemID'].count()

userID
0       10180
1       10179
2       10177
3       10182
4       10182
        ...  
1995    10177
1996    10180
1997    10180
1998    10180
1999    10179
Name: itemID, Length: 2000, dtype: int64

In [ ]:
all_predictions['prediction'].max()

0.056523170322179794

In [ ]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_rmse = rmse(test, all_predictions, col_prediction='prediction')

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall,
      "RMSE :\t%f" % eval_rmse, sep='\n')

MAP:	0.002612
NDCG:	0.005472
Precision@K:	0.002449
Recall@K:	0.008626
RMSE :	3.874406


In [ ]:
#https://habr.com/ru/company/econtenta/blog/303458/ статья на тему метрик

In [33]:
# Record results with papermill for tests
sb.glue("map", eval_map)
sb.glue("ndcg", eval_ndcg)
sb.glue("precision", eval_precision)
sb.glue("recall", eval_recall)

---

In [2]:
#pip install "pymanopt@https://github.com/pymanopt/pymanopt/archive/fb36a272cdeecb21992cfd9271eb82baafeb316d.zip"